# Fast annotation toolkit

## Install dependencies

In [13]:
!pip install -q jupyter_bbox_widget

In [15]:
!apt-get -yq install imagemagick

Reading package lists...
Building dependency tree...
Reading state information...
imagemagick is already the newest version (8:6.9.11.60+dfsg-1.3ubuntu0.22.04.3).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.


## Imports

In [21]:
from jupyter_bbox_widget import BBoxWidget
import ipywidgets as widgets
import os
import json
import base64
import uuid

In [ ]:
# Run this if you use Colab
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
def encode_image(filepath):
    with open(filepath, 'rb') as f:
        image_bytes = f.read()
    encoded = str(base64.b64encode(image_bytes), 'utf-8')
    return "data:image/jpg;base64,"+encoded

## Conventions

Input images should be dropped in the "images/" directory.

Resized images will go in the "images_resized/" directory.

Annotations (single csv file) will be named "annotations.csv".

Files will be named with a random value to avoid collisions.

## Convert the images

In [72]:
input_dir = "images/"
output_dir = "images_ready/"
annotations_file = "annotations.csv"

class_names = ["logo", ]

In [30]:
# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

files = sorted(os.listdir(input_dir))

for file in files:
    if file.endswith((".jpg", ".jpeg", ".png")):  # Adjust the extensions as needed
        input_path = os.path.join(input_dir, file)

        # Generate a unique random filename with a .png extension
        unique_filename = str(uuid.uuid4()) + ".png"
        output_path = os.path.join(output_dir, unique_filename)

        # Use the convert command to resize the image and specify PNG as the output format
        !convert "{input_path}" -resize 256x256^ -gravity center -extent 256x256 "{output_path}"

In [31]:
!identify "{output_dir}"/*

images_ready//1eede26f-0ff3-4a0a-bf97-97cdc8678dea.png PNG 256x256 256x256+0+0 8-bit sRGB 132365B 0.000u 0:00.000
images_ready//20e40369-56e4-41f0-837a-877c7e8c68b1.png PNG 256x256 256x256+0+0 8-bit sRGB 171563B 0.000u 0:00.000
images_ready//b0ec626a-1e0c-4b7f-ad76-ab80ac6d06a7.png PNG 256x256 256x256+0+0 8-bit sRGB 130060B 0.000u 0:00.000
images_ready//e8b6fc7c-e2e9-422d-a500-c210986890c6.png PNG 256x256 256x256+0+0 8-bit sRGB 191449B 0.000u 0:00.000


## Prepare UI

In [60]:
# Images are in the images directory
image_path = output_dir
files = sorted(os.listdir(image_path))

# a progress bar to show how far we got
w_progress = widgets.IntProgress(value=0, max=len(files), description='Progress')
# the bbox widget
w_bbox = BBoxWidget(
    image = encode_image(os.path.join(image_path, files[0])),
    classes=class_names
)

# combine widgets into a container
w_container = widgets.VBox([
    w_progress,
    w_bbox,
])

Define the functions to process clicks on our Submit and Skip buttons.

You can use the widget's `on_skip` and `on_submit` methods as decorators on the functions.

In [61]:
# start with empty annotation list
annotations = {}

In [62]:
# when Skip button is pressed we move on to the next file
@w_bbox.on_skip
def skip():
    if w_progress.value + 1 == len(files):
      print("Finished!")
      return
    w_progress.value += 1
    # open new image in the widget
    image_file = files[w_progress.value]
    w_bbox.image = encode_image(os.path.join(image_path, image_file))
    # here we assign an empty list to bboxes but
    # we could also run a detection model on the file
    # and use its output for creating inital bboxes
    w_bbox.bboxes = []

# when Submit button is pressed we save current annotations
# and then move on to the next file
@w_bbox.on_submit
def submit():
    image_file = files[w_progress.value]
    # save annotations for current image
    annotations[image_file] = w_bbox.bboxes
    # with open(annotations_path, 'w') as f:
    #     json.dump(annotations, f, indent=4)
    # move on to the next file
    skip()

## Run the annotation

In [63]:
w_container

Finished!


## Control and export

In [64]:
# Display the annotations
annotations

{'1eede26f-0ff3-4a0a-bf97-97cdc8678dea.png': [{'x': 66,
   'y': 93,
   'width': 54,
   'height': 28,
   'label': 'logo'}],
 '20e40369-56e4-41f0-837a-877c7e8c68b1.png': [{'x': 110,
   'y': 131,
   'width': 38,
   'height': 31,
   'label': 'logo'}],
 'b0ec626a-1e0c-4b7f-ad76-ab80ac6d06a7.png': [{'x': 143,
   'y': 34,
   'width': 41,
   'height': 22,
   'label': 'logo'}],
 'e8b6fc7c-e2e9-422d-a500-c210986890c6.png': [{'x': 48,
   'y': 203,
   'width': 31,
   'height': 18,
   'label': 'logo'}]}

In [70]:
# Make sure we have at most one annotation per image
for image_file, bboxes in annotations.items():
  if len(bboxes) != 1:
    print(f"ERROR: {image_file} has {len(bboxes)} bboxes, expected 1. Please fix this image before exporting.")
    continue
  if bboxes[0]["label"] != "logo":
    print(f"ERROR: {image_file} has an unkowned label {bboxes[0]['label']}. Please fix this image before exporting.")

In [71]:
import csv

def export_annotations_to_csv(data, csv_file):

    with open(csv_file, 'w', newline='') as csv_output:
        csv_writer = csv.writer(csv_output)
        csv_writer.writerow(['img_name', 'class_id', 'x1', 'x2', 'y1', 'y2'])

        for img_name, annotations in data.items():
            annotation = annotations[0]  # Assuming there's only one logo object per image

            x1 = annotation['x']
            y1 = annotation['y']
            x2 = x1 + annotation['width']
            y2 = y1 + annotation['height']
            label = 0

            # Write the data to the CSV file
            csv_writer.writerow([img_name, label, x1, x2, y1, y2])

In [73]:
export_annotations_to_csv(annotations, annotations_file)

In [74]:
!cat "{annotations_file}"

img_name,class_id,x1,x2,y1,y2
1eede26f-0ff3-4a0a-bf97-97cdc8678dea.png,0,66,120,93,121
20e40369-56e4-41f0-837a-877c7e8c68b1.png,0,110,148,131,162
b0ec626a-1e0c-4b7f-ad76-ab80ac6d06a7.png,0,143,184,34,56
e8b6fc7c-e2e9-422d-a500-c210986890c6.png,0,48,79,203,221


## Package for submission

In [76]:
submission_file = "sub.tar.gz"

In [77]:
!tar czf "{submission_file}" "{annotations_file}" "{output_dir}"
!ls -lA "{submission_file}"

-rw-r--r-- 1 root root 624880 Oct 25 09:21 submission_file


## Download your file locally

In [78]:
from google.colab import files
files.download(submission_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Now submit this file on Moodle.